In [3]:
from datasets import load_from_disk, Dataset
import os

In [9]:
import torch

In [12]:
torch.device

torch.device

In [13]:
train = load_from_disk(os.path.join("./data/KorQuAD-v1.0"))['train']

In [14]:
train[0]

{'answers': {'text': ['교향곡'], 'answer_start': [54]},
 'context': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.',
 'id': '6566495-0-0',
 'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?',
 'title': '파우스트_서곡'}

In [19]:
train

Dataset({
    features: ['answers', 'context', 'id', 'question', 'title'],
    num_rows: 60407
})

In [15]:
import json

In [16]:
with open("./data/KorQuAD-v1.0/wiki_800.json", "r", encoding="utf-8") as f:
            corpus = json.load(f)

In [20]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, BertForTokenClassification, pipeline

In [25]:
def get_qa_model():
    model = AutoModelForQuestionAnswering.from_pretrained("./outputs/koelectra_v3_test1")
    tokenizer = AutoTokenizer.from_pretrained("./outputs/koelectra_v3_test1")
    model.to(torch.device('cuda:0'))
    qa = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0)
    return qa

In [26]:
qa = get_qa_model()

In [253]:
context = """
여행·숙박 정보 애플리케이션(앱) ‘야놀자’가 자사 숙박업소 정보를 빼낸 경쟁사 ‘여기어때’를 상대로 제기한 민사소송 1심에서 승소했다.

23일 법원에 따르면 서울중앙지법 민사합의63-2부(부장 박태일 이민수 이태웅)는 야놀자 측이 여기어때 운영사 여기어때컴퍼니를 상대로 제기한 권리침해금지 소송에서 “여기어때 측은 10억 원을 지급하라”며 원고 일부승소 판결했다.

앞서 야놀자는 2016년 자사 서버에 접속이 몰려 장애가 발생하자 원인 분석에 나섰다. 자체 조사 결과, 경쟁사인 여기어때가 숙박업소 정보를 대량으로 탈취하려고 한 정황을 파악, 법적 대응에 나섰다.

실제 여기어때는 2015년부터 야놀자의 제휴 숙박업소 목록, 주소·가격 정보 등을 수기로 취합해 내부적으로 공유해 온 것으로 조사됐다. 2016년부터는 아예 크롤링(검색 엔진 로봇을 이용한 자동 데이터 수집 방법) 프로그램을 개발해, 야놀자 전산 서버에 1,500만여회 이상 접속해 대량의 정보를 무단 복제했고 이를 영업전략 수립 등에 사용했다.


법원은 여기어때가 야놀자가 축적한 데이터를 무단으로 사용해 경제적인 이익을 얻었다고 판단했다. 재판부는 "야놀자가 상당한 비용과 시간을 들여서 수집·분류·갱신한 정보가 무제한적으로 유출될 경우 원고의 경쟁력이 저하되는 손해가 발생할 수 있다"고 밝혔다. 이어 "여기어때는 이 같은 정보 수집 방식이 일반적이고 정당한 행위라고 주장하지만, 자사 서버를 쓰는 것은 발각될 우려가 있다고 여겨 크롤링 프로그램을 다른 웹사이트 클라우드에 이전해 설치한 사실을 고려하면 문제가 된다는 사실을 알고 있었다고 보인다“고 지적했다.

여기어때 심모 전 대표 등 관계자들은 정보통신망법 위반 등 혐의로 2019년 3월 기소돼 1심에서 유죄를 선고받았으나, 올해 1월 항소심에선 무죄 판결을 받았다. 당시 항소심 재판부는 “(무단 수집된 정보가) 적극적으로 공개되진 않았지만, 피해자 회사(야놀자)가 이를 비공개하거나 숨긴 것으로 보이진 않는다”고 밝혔다. 다만 “피해자 회사는 선두주자로서 인터넷 숙박영업을 활성화하는 데 투자·노력·시간을 들인 것으로 충분히 짐작되고, 후발 주자인 피고인들(여기어때 측)로서는 그 노력을 상당히 줄이고 사업을 단시간에 정상화하려 했다고 짐작된다"고 꼬집었다.

"""

In [254]:
context = context.strip()
context = context.replace('\n\n'," ")
context = context.replace('\n'," ")

In [255]:
context

'여행·숙박 정보 애플리케이션(앱) ‘야놀자’가 자사 숙박업소 정보를 빼낸 경쟁사 ‘여기어때’를 상대로 제기한 민사소송 1심에서 승소했다. 23일 법원에 따르면 서울중앙지법 민사합의63-2부(부장 박태일 이민수 이태웅)는 야놀자 측이 여기어때 운영사 여기어때컴퍼니를 상대로 제기한 권리침해금지 소송에서 “여기어때 측은 10억 원을 지급하라”며 원고 일부승소 판결했다. 앞서 야놀자는 2016년 자사 서버에 접속이 몰려 장애가 발생하자 원인 분석에 나섰다. 자체 조사 결과, 경쟁사인 여기어때가 숙박업소 정보를 대량으로 탈취하려고 한 정황을 파악, 법적 대응에 나섰다. 실제 여기어때는 2015년부터 야놀자의 제휴 숙박업소 목록, 주소·가격 정보 등을 수기로 취합해 내부적으로 공유해 온 것으로 조사됐다. 2016년부터는 아예 크롤링(검색 엔진 로봇을 이용한 자동 데이터 수집 방법) 프로그램을 개발해, 야놀자 전산 서버에 1,500만여회 이상 접속해 대량의 정보를 무단 복제했고 이를 영업전략 수립 등에 사용했다.  법원은 여기어때가 야놀자가 축적한 데이터를 무단으로 사용해 경제적인 이익을 얻었다고 판단했다. 재판부는 "야놀자가 상당한 비용과 시간을 들여서 수집·분류·갱신한 정보가 무제한적으로 유출될 경우 원고의 경쟁력이 저하되는 손해가 발생할 수 있다"고 밝혔다. 이어 "여기어때는 이 같은 정보 수집 방식이 일반적이고 정당한 행위라고 주장하지만, 자사 서버를 쓰는 것은 발각될 우려가 있다고 여겨 크롤링 프로그램을 다른 웹사이트 클라우드에 이전해 설치한 사실을 고려하면 문제가 된다는 사실을 알고 있었다고 보인다“고 지적했다. 여기어때 심모 전 대표 등 관계자들은 정보통신망법 위반 등 혐의로 2019년 3월 기소돼 1심에서 유죄를 선고받았으나, 올해 1월 항소심에선 무죄 판결을 받았다. 당시 항소심 재판부는 “(무단 수집된 정보가) 적극적으로 공개되진 않았지만, 피해자 회사(야놀자)가 이를 비공개하거나 숨긴 것으로 보이진 않는다”고 밝혔다. 다만 “피해자 회사는 선두주자로서

In [259]:
question = "소송에서 어느 회사가 승소했어?"

In [260]:
ans = qa(question=question, context=context, topk=5)

In [261]:
ans

[{'score': 0.25960373878479004, 'start': 46, 'end': 50, 'answer': '여기어때'},
 {'score': 0.13241702318191528, 'start': 522, 'end': 526, 'answer': '야놀자가'},
 {'score': 0.11960092186927795, 'start': 20, 'end': 23, 'answer': '야놀자'},
 {'score': 0.09412376582622528, 'start': 936, 'end': 942, 'answer': '피해자 회사'},
 {'score': 0.017827479168772697, 'start': 936, 'end': 939, 'answer': '피해자'}]

In [211]:
context

'닭가슴살이 서로 겹치지 않게 냄비 바닥에 깐다. 닭가슴살이 들어갈 정도로 큰 냄비가 없다면 깊고 뚜껑이 있는 프라이팬을 사용한다.냉동 닭가슴살은 완전히 해동시킨다. 닭가슴살이 냉동되어 있으면 조리 시간에 영향을 미치며 골고루 익지 않을 수도 있다. 닭가슴살이 여전히 얼어 있으면 냉장고에 넣어 해동시킨 후 익힌다. 냄비에 물을 7 cm 가량 붓고 필요하면 더 부어서 닭가슴살이 물에 잠기게 한다. 냄비에 물을 너무 많이 채우지 않도록 주의한다. 닭가슴살을 익히는 동안 물이 끓어 넘치기 때문이다. 이를 방지하기 위해 5 cm 가량 비워둔다.닭가슴살에 양념을 따로 하지 않는다. 개가 배탈이 나는 원인이 되기 때문이다. 닭가슴살을 담백하게 삶고 원한다면 나중에 다른 먹이에 곁들인다.[4]X신뢰도 있는 출처American Kennel Club출처 보기 강불로 물을 끓인다. 그 다음에 계속해서 닭가슴살을 12분 가량 삶는다.12분 후 닭가슴살 하나를 냄비에서 꺼내 잘라서 다 익었는지 확인한다. 닭가슴살 속이 여전히 분홍색이거나 고무 같으면 다시 냄비에 넣고 계속 1-2분 더 익힌다. 나이프와 포크를 사용하거나 포크 2개를 사용하면 닭가슴살을 쉽게 찢을 수 있다. 개가 씹어 삼키기 좋게 잘게 찢는다.[6]X신뢰도 있는 출처American Kennel Club출처 보기닭가슴살을 찢을 때 개의 크기를 염두에 둔다. 소형견은 대형견에 비해 닭가슴살을 더 잘게 찢어줘야 한다. 닭가슴살을 만지면 차가울 때까지 조리대 위에 놓아둔다. 닭가슴살이 식으면 개에게 간식으로 주거나 다른 먹이에 곁들여 식사로 준다.닭가슴살을 냉장고에 5분 동안 넣어두면 더 빠르게 식힐 수 있다. 개에게 훈련에 대한 보상이나 간식으로 닭가슴살을 준다. 개에게 삶은 닭가슴살을 너무 많이 먹이지 않는다.닭가슴살을 훈련 도구로 이용하는 경우 개가 과제에 성공하면 잘게 찢은 닭가슴살을 한 점 준다.닭가슴살을 간식으로 주는 경우 개의 크기에 따라 적당한 양을 준다. 개에게 식사로 먹이를 얼마나 주는지 생각해보고 여기에

In [60]:
import re

In [39]:
import pickle

In [47]:
with open("data/df_for_category.pickle", "rb") as f:
    data = pickle.load(f)

In [52]:
dog = data[data.category == "개"].reset_index(drop=True)

In [208]:
context = dog.iloc[20,:]['contexts']

In [209]:
context = ' '.join(context)

In [210]:
context = re.sub(r'<.+>', "", context)
context = re.sub(r'\n', "", context)
context = re.sub(r'\s+', " ", context).strip()

In [207]:
dog.iloc[20,:]

category                                                       개
main_title                                      개에게 닭고기를 삶아주는 방법
context_url    https://ko.wikihow.com/%EA%B0%9C%EC%97%90%EA%B...
is_expert                                                    전문가
contexts       [<sub>닭가슴살 익히기</sub> <b>뼈와 껍질이 없는 닭가슴살 3개를 중간 ...
tips                                                     no_tips
warnings       잘게 찢은 삶은 닭가슴살을 완전히 식혀서 개에게 먹여야 한다. 그렇지 않으면 개가 ...
Name: 20, dtype: object

In [172]:
dog.iloc[10,:].context_url

'https://ko.wikihow.com/%EA%B0%9C-%EC%93%B0%EB%8B%A4%EB%93%AC%EB%8A%94-%EB%B0%A9%EB%B2%95'